In [1]:
import os
from typing import Any, Type
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
import requests

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo-0125",
)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to fine the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock market Symbol for Apple Company
    """

    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company. Example:APPL, TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the finalcials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the daily performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock, considering it's finalcials, income statements, stock performance and help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock market Symbol for Cloudflare Company'}`


Is Cloudflare stock a Buy, Sell or Hold? What was the 52-week low for Cloudflare stock? What was the 52-week high for Cloudflare stock? What are analysts forecasts for Cloudflare... Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. ... New York (USD) · Market . closed +1.52 +1.58 % 99.80 -0.12 (-0.12%) After-hours: Mar 1, 2024, 7:59 PM EST Financials Statistics Profile Chart YTD 1Y 5Y Max 1.40% ( 1D) About NET Employees Stock Exchange Ticker Symbol Financial Performance In 2023, Cloudflare's revenue was $1.30 billion, an increase of 32.97% compared to the previous year's $975.24 million. Stocks Cloudflare Inc Cloudflare Inc NET Morningstar Rating Unlock Stock XNYS Rating as of Feb 27, 2024 Summary Chart News Price vs Fair Value Sustainability Tr

{'input': "Give me information on Cloudflare's stock, considering it's finalcials, income statements, stock performance and help me analyze if it's a potential good investment.",
 'output': "### Cloudflare Inc. Overview:\n- **Symbol:** NET\n- **Description:** CloudFlare, Inc. operates a cloud platform that offers a range of network services to companies around the world. \n- **Sector:** Technology\n- **Industry:** Services-Prepackaged Software\n- **Market Capitalization:** $34.02 billion\n- **52-Week High:** $116\n- **52-Week Low:** $39.9\n- **Current Price:** $97.99\n\n### Financials:\n- **Revenue (TTM):** $1.30 billion\n- **Gross Profit (TTM):** $742.63 million\n- **Net Income (TTM):** -$183.95 million\n- **EPS (TTM):** -$0.55\n- **Profit Margin:** -14.2%\n- **Operating Margin (TTM):** -11.8%\n- **Return on Assets (TTM):** -4.34%\n- **Return on Equity (TTM):** -26.5%\n\n### Analysts Forecast:\n- **Average Target Price:** $101.73\n- **Analyst Ratings:** 5 Strong Buy, 10 Buy, 14 Hold, 